In [34]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from agents import OpenAIChatCompletionsModel
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from dotenv import load_dotenv
from openai import AsyncOpenAI
from langchain.agents import initialize_agent
import os
load_dotenv(override=True)

True

In [35]:
client = AsyncOpenAI(
    api_key=os.getenv("GEMINI_ACCESS_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)

In [36]:
# Gemini for reasoning/answer generation
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001",google_api_key=os.getenv('GEMINI_ACCESS_KEY'))  # or "gemini-1.5-pro"

# Gemini embeddings for vector store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.getenv('GEMINI_ACCESS_KEY'))

In [37]:
urls = [
    "https://python.langchain.com/docs/",
    "https://qdrant.tech/documentation/",
    "https://blog.langchain.com/agentic-rag-with-langgraph/"
]

In [38]:
# Load & split
# docs = [WebBaseLoader(url).load()[0] for url in urls]
docs = []
for url in urls:
    loaded = WebBaseLoader(url).load()
    docs.extend(loaded) 

docs_list = docs
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=50)
# splits = splitter.split_documents(docs_list)
splits = splitter.split_documents(docs)

# Extract only text from the documents
texts = [doc.page_content for doc in splits]

# Now embed only the text
vectorstore = InMemoryVectorStore.from_texts(texts, embedding=embeddings)
retriever = vectorstore.as_retriever()

def retrieve_docs(query: str) -> str:
    """Retrieve relevant docs from the vectorstore"""
    results = retriever.get_relevant_documents(query)
    return "\n\n".join([doc.page_content for doc in results])

retriever_tool = Tool(
    name="retrieve",
    func=retrieve_docs,
    description="Retrieve relevant documents given a user query"
)

tools = [retriever_tool]


In [39]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="openai-functions",
    verbose=True
)

result = agent.run("What is langChain")
print("Agentic RAG Answer:", result)




> Entering new AgentExecutor chain...
I need to find out what LangChain is. A good way to do that is to use the retrieve tool to search for information about it.
Action: retrieve
Action Input: "What is LangChain?"
Observation: Introduction | ü¶úÔ∏èüîó LangChain

LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Join our newsletter
Updates from the LangChain team and community


Enter your email

Subscribe

Processing your application...
Success! Please check your inbox and click the link to confirm your subscription.
Sorry, something went wrong. Please try again.













Sign up





            Â© LangChain Blog 2025

Self-Reflective RAG with LangGraph

















































Skip to content
















Case Studies




In the Loop




LangChain




Docs




Changelog





Sign in
Subscribe



















Self-Reflective RAG with Lan

In [40]:
result = agent.run("What is gRPC")
print("Agentic RAG Answer:", result)



> Entering new AgentExecutor chain...
I should find out what gRPC is.
Action: retrieve
Action Input: "what is gRPC"
Observation: Introduction | ü¶úÔ∏èüîó LangChain

Join our newsletter
Updates from the LangChain team and community


Enter your email

Subscribe

Processing your application...
Success! Please check your inbox and click the link to confirm your subscription.
Sorry, something went wrong. Please try again.













Sign up





            Â© LangChain Blog 2025

Self-Reflective RAG with LangGraph

















































Skip to content
















Case Studies




In the Loop




LangChain




Docs




Changelog





Sign in
Subscribe



















Self-Reflective RAG with LangGraph

6 min read
Feb 7, 2024

LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:
Thought:The retrieved document does not define gRPC. I need to retr